In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import os, sys, numpy as np
import re
import torch

#Parse data into testing and training examples
def parse_data(line):
    data = []
    
    #If line starts with a quotation mark, then there is a comma in the data and rsplit must be used
    if line.startswith("\""):
        #find the ending quotation mark to create msg substring
        data = line.rsplit(",", 4)
    else:
        data = line.split(",")
    
    msg = data.pop(0)
    quantifiers = data
    return msg, quantifiers

def get_Ngrams(corpora, scores, ngrams=2):
    X = []
    Y = []
    word_regx = re.compile('^[a-z]*$')
    for i in range(len(corpora)):
        words = corpora[i].split(" ")
        rs = [] #regex'd sentence

        #regex sentence
        for w in words:
            w = re.sub('[.,!?\\-()\"]', "", w).lower()
            match = word_regx.match(w)
            if match != None:
                rs.append(w)

        #make ngram out of sentence and add to output
        if len(rs) < ngrams: #only use sentence if it has enough words for the ngram
            continue
        #create list of ngrams from sentence and also the cooresponding scores
        #found the zip function from here: https://stackoverflow.com/questions/13423919/computing-n-grams-using-python
        for j in zip(*[rs[i:] for i in range(ngrams)]):
            X.append(j)
            Y.append(scores[i])

    return np.asarray(X), np.asarray(Y)


#Parse data    
data = open("dataset-fb-valence-arousal-anon.csv", encoding="utf8")
data.readline()
corpora = []
scores = [] 
for line in data.readlines():
    msg, quantifiers = parse_data(line)
    v1, v2, a1, a2 = quantifiers
    corpora.append(msg)
    scores.append((int(v1), int(a1)))

#get the ngrams
X, Y = get_Ngrams(corpora, scores, 3)

print("X: ", X.shape)
print("Y: ", Y.shape)


X:  (39657, 3)
Y:  (39657, 2)
